In [1]:
gROOT->ProcessLine(".L common/gallery_includes.h+O");
gROOT->ProcessLine(".L common/larsoftobj_includes.h+O");
gROOT->ProcessLine(".L common/My_ROOT_Utilities.h");

In [2]:
//for space charge
gROOT->ProcessLine(".L $GSL_LIB/libgsl.so");
gROOT->ProcessLine(".L $UBOONECODE_LIB/libuboone_SpaceCharge.so");
gROOT->ProcessLine(".L common/Utils_SpaceCharge.h+O")

(long) 0


In [3]:
//setup our input file and input tag
std::vector<std::string> filenames;
std::string file_path;
TFile *f_output;

TNtuple *nt;
TCanvas canvas("canvas","MyCanvas");

art::InputTag vtx_tag;
art::InputTag pfp_tag;
art::InputTag gen_tag;

//geometry
float X_LENGTH=256.0;
float Y_HALF_HEIGHT=116.5;
float Z_LENGTH=1037.0;

float PROTON_THRESHOLD=0.030;

std::unique_ptr<spacecharge::SpaceChargeMicroBooNE> sce;
CreateSCE(sce);
geo::Vector_t offsets;

bool verbose=false;

In [4]:
%%cpp -d
void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const&,
                    int ntab=0);

void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const& pfp_vec,
                    int ntab)
{
    for(size_t i_d=0; i_d<pfp_vec[i_pfp].NumDaughters(); ++i_d){
        for(int it=0; it<ntab; ++it) std::cout << "\t";
        std::cout << "Found daughter particle id=" << pfp_vec[i_pfp].Daughters()[i_d] 
                  << ", code=" << pfp_vec[pfp_vec[i_pfp].Daughters()[i_d]].PdgCode() << std::endl;
        PrintDaughters(pfp_vec[i_pfp].Daughters()[i_d],pfp_vec,ntab+1);
    }
    return;
}

In [5]:
verbose=false;

//filepath = "/pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/";
filepath = "";
//filenames = GetFileNameList("/uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_devset_reco2_stripped_filelist.txt",filepath);
filenames = GetFileNameList("/uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nue_tune3_reco2_stripped_filelist.txt",filepath);

f_output = new TFile("/uboone/data/users/wketchum/nue_tune3_vtx_ntuple_trj_tmp.root","RECREATE");

nt = new TNtuple("nt","MyNtuple","nu_x:nu_y:nu_z:nu_e:q2:lep_p:n_p:p_ke:vtx_x:vtx_y:vtx_z:vtx_dist:pfp_primary:pfp_pdg");

vtx_tag = { "trajcluster" };
pfp_tag = { "trajcluster" };
gen_tag = { "generator" };


In [6]:
//filenames.size()
//std::cout << "working?" << std::endl;
//for(auto & fname : filenames)
    //std::cout << fname << std::endl;
//filenames.resize(1);

In [7]:
nt->Reset();
for(gallery::Event ev(filenames) ; !ev.atEnd(); ev.next()) {
    if(verbose)
        std::cout << "Processing " 
                  << "Run " << ev.eventAuxiliary().run() << ", "
                  << "Event " << ev.eventAuxiliary().event() << endl;
    
    auto const& mctruth_handle = ev.getValidHandle<std::vector<simb::MCTruth>>(gen_tag);
    auto const& truth_vec(*mctruth_handle);

    if(truth_vec.size()==0){
        std::cout << "ERROR! truth_vec size is " << truth_vec.size() << std::endl;
        break;
    }
       
    auto const& nu = truth_vec[0].GetNeutrino();
    geo::Point_t nu_position(nu.Nu().Vx(),nu.Nu().Vy(),nu.Nu().Vz());

    if(verbose)
        std::cout << "Neutrino position before sce correction is (" 
                  << nu_position.X() << ","
                  << nu_position.Y() << ","
                  << nu_position.Z() << ")"
                  << std::endl;

    if(nu_position.X()<0.0+10.0 || nu_position.X()>X_LENGTH-10.0 ||
       nu_position.Y()<-1*Y_HALF_HEIGHT+20.0 || nu_position.Y()>Y_HALF_HEIGHT-20.0 ||
       nu_position.Z()<0.0 || nu_position.Z()>Z_LENGTH-10.0 )
        continue;

    nu_position = nu_position + sce->GetPosOffsets(nu_position);

    if(verbose)
        std::cout << "Neutrino position after sce correction is (" 
                  << nu_position.X() << ","
                  << nu_position.Y() << ","
                  << nu_position.Z() << ")"
                  << std::endl;

    float lep_energy = -999, lep_p = -999;
    if(nu.CCNC()==0) { //CC
        lep_energy = nu.Lepton().E();
        lep_p = nu.Lepton().P();
    }
    int n_proton=0; float max_proton_ke=0.0;

    for(size_t i_p=0; i_p<truth_vec[0].NParticles(); ++i_p){
        auto const& p = truth_vec[0].GetParticle(i_p);

        if(p.StatusCode()!=1 || p.PdgCode()!=2212) continue;
        
        auto ke = (p.E()-p.Mass());
        if(ke > PROTON_THRESHOLD) ++n_proton;
        if(ke>max_proton_ke) max_proton_ke = ke;        
    }

    auto const& pfp_handle = ev.getValidHandle<std::vector<recob::PFParticle>>(pfp_tag);
    auto const& pfp_vec(*pfp_handle);
    if(verbose)
        std::cout << "There are " << pfp_vec.size() << " PFParticles in this event." << std::endl;

    for (auto const& pfp : pfp_vec){
        if(!pfp.IsPrimary()) continue;
        if(verbose){
            std::cout << "Found primary particle id=" << pfp.Self() << ", code=" << pfp.PdgCode() << std::endl;
            PrintDaughters(pfp.Self(),pfp_vec,1);
        }
    }
    
    
    auto const& vtx_handle = ev.getValidHandle<std::vector<recob::Vertex>>(vtx_tag);
    auto const& vtx_vec(*vtx_handle);
    if(verbose)
        std::cout << "There are " << vtx_vec.size() << " vertices in this event." << std::endl;

    art::FindOneP<recob::Vertex> vtx_per_pfp(pfp_handle,ev,pfp_tag);
    art::Ptr<recob::Vertex> vtxptr;
    double min_dist=9e9; int i_pfp_nu=-1; geo::Point_t pfp_position;
    
    for(size_t i_pfp=0; i_pfp<pfp_vec.size(); ++i_pfp)
    {
        
        vtx_per_pfp.get(i_pfp,vtxptr);
        if(vtxptr.isNonnull()){
            auto const& vtx(*vtxptr);
            if(verbose)
                std::cout << "For pfparticle " << i_pfp << " found vertex at "
                          << "(" << vtx.position().X() << ","
                          << ""  << vtx.position().Y() << ","
                          << ""  << vtx.position().Z() << ")"
                          << std::endl;
            
            if(sqrt((vtx.position()-nu_position).Mag2()) < min_dist){
                min_dist = sqrt((vtx.position()-nu_position).Mag2());
                i_pfp_nu = i_pfp;
                pfp_position = vtx.position();
            }
        }
        
            
    }
    
    nt->Fill(nu_position.X(),nu_position.Y(),nu_position.Z(),
             nu.Nu().E(),nu.QSqr(),lep_p,n_proton,max_proton_ke,
             pfp_position.X(),pfp_position.Y(),pfp_position.Z(),
             min_dist,
             pfp_vec[i_pfp_nu].IsPrimary(),pfp_vec[i_pfp_nu].PdgCode());
    
    
}//end loop over events


Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_1.root


%MSG-d ProductID: 
Product created with id: [3130632973] from canonical product name: "simb::MCTruths_generator__GenieGen."
%MSG
%MSG-d ProductID: 
Product created with id: [4283826371] from canonical product name: "simb::MCTruths_generator__G4."
%MSG
%MSG-d ProductID: 
Product created with id: [852458230] from canonical product name: "simb::MCTruths_generator__Detsim."
%MSG
%MSG-d ProductID: 
Product created with id: [2024314490] from canonical product name: "simb::MCTruths_generator__McRecoStage1."
%MSG
%MSG-d ProductID: 
Product created with id: [1401275833] from canonical product name: "simb::MCTruths_generator__McRecoStage2."
%MSG
%MSG-d ProductID: 
Product created with id: [455092605] from canonical product name: "simb::MCTruths_generator__Strip."
%MSG
%MSG-d ProductID: 
Product created with id: [2571112662] from canonical product name: "recob::PFParticles_trajcluster__GenieGen."
%MSG
%MSG-d ProductID: 
Product created with id: [3939597955] from canonical product name: "recob::PF

Closing file, read 2249642 bytes in 521 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_10.root
Closing file, read 2237897 bytes in 450 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_100.root
Closing file, read 2620355 bytes in 431 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_101.root
Closing file, read 2661641 bytes in 446 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_102.root
Closing file, read 2637857 bytes in 437 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_103.root
Closing file, read 2643451 bytes in 428 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3

Closing file, read 2834749 bytes in 435 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_143.root
Closing file, read 2843652 bytes in 441 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_144.root
Closing file, read 2838223 bytes in 430 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_145.root
Closing file, read 2857353 bytes in 458 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_146.root
Closing file, read 2848882 bytes in 440 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_147.root
Closing file, read 2822905 bytes in 424 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune

Closing file, read 3069496 bytes in 459 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_187.root
Closing file, read 3042370 bytes in 459 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_188.root
Closing file, read 3084601 bytes in 456 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_189.root
Closing file, read 3067684 bytes in 449 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_19.root
Closing file, read 2272644 bytes in 452 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_190.root
Closing file, read 4127294 bytes in 434 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3

Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_55.root
Closing file, read 2440023 bytes in 451 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_56.root
Closing file, read 2387501 bytes in 428 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_57.root
Closing file, read 2450811 bytes in 437 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_58.root
Closing file, read 2453005 bytes in 467 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_59.root
Closing file, read 2442459 bytes in 422 transactions
Successfully opened file /uboone/data/users/wketchum/nue_tune3_reco2_stripped_files/nu3_tune3_reco2_stripped_6.root
Closing file, read 2179123 bytes i

Closing file, read 2642605 bytes in 456 transactions


In [8]:
f_output->cd();
nt->Write();
f_output->Close();

Todos:

---Double-check space charge correction.

---Store lepton angles and number of final state particles in tree

---

---Understand the fraction of charge in the reconstructed heirarchies relative to the truth deposited. Compare across "well-reconstructed" (close to vertex) and "poorly-reconstructed" (not close to vertex)

---Should look at distance of reconstructed primary to the truth, and not restrict to closest vertex (could be a daughter is super close to the primary too)

